# IA para espacialização dos dados

In [1]:
import os
import rasterio
import numpy as np
import pandas as pd
import asciichartpy
import seaborn as sns
import geopandas as gpd
import rioxarray as rxr
import matplotlib.pyplot as plt

from tqdm import tqdm, trange
from pyproj import Transformer
from shapely import Point, distance
from rasterio.transform import rowcol
from IPython.display import clear_output
from utils.consts import SOIL_TYPES, USO_SOLO_CLASS

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, CyclicLR
from torch.utils.data import TensorDataset, Dataset, DataLoader, random_split

### Device disponível para treinar o modelo MLP

In [2]:
# 1) Dispositivo (GPU se disponível)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


### Dataset com os dados para convolução
---
- O Dataset nesta aplicação é extremamente importante pois irá englobar todas as informações necessárias para espacializar os dado de infiltração

In [ ]:
class MeuDataset(Dataset):
    nix_bands = [
        "R400 nm", "R410 nm", "R420 nm", "R430 nm",
        "R440 nm", "R450 nm", "R460 nm", "R470 nm",
        "R480 nm", "R490 nm", "R500 nm", "R510 nm",
        "R520 nm", "R530 nm", "R540 nm", "R550 nm",
        "R560 nm", "R570 nm", "R580 nm", "R590 nm",
        "R600 nm", "R610 nm", "R620 nm", "R630 nm",
        "R640 nm", "R650 nm", "R660 nm", "R670 nm",
        "R680 nm", "R690 nm", "R700 nm",
    ]

    columns_dados = ["Ponto", "Lat", "Lon", "soils_type", "Clay", "Silt", "Sand", "K (C1)"]

    columns_uso_solo = [1, 2, 5, 6, 8, 9]

    def __init__(self, device:torch.device|None=None, eval=False):
        """
        O dataset tem o formato de uma tupla com os valores em X e em Y:
        - (X, Y)
        - Onde:
        - X: (rasters_vals, [Dist. Talvegue, Tipo do Solo, na ordem de soil_types])
        - Y: K (cm/s)
        """
        self.eval = eval
        self.device = device
        self.janela = 15 # Janela deve ser ímpar

        if self.janela%2 == 0:
            raise ValueError("A janela deve ser ímpar")

        # Lendo Tabelas
        print("Lendo Tabelas")
        self.dados = pd.read_excel(r"D:\Mestrado\Trabalho Final\Dados\Levantamento em Campo\Compiled.xlsx", sheet_name="Infiltracao")
        self.dados = self.dados[self.columns_dados].dropna().reset_index(drop=True)
        gdf = gpd.GeoDataFrame(self.dados, geometry=gpd.points_from_xy(self.dados["Lat"], self.dados["Lon"]), crs="EPSG:4326")
        gdf.to_crs("EPSG:31983", inplace=True)
        self.dados['Lat'] = gdf.geometry.y
        self.dados['Lon'] = gdf.geometry.x

        self.nix = pd.read_excel(r"D:\Mestrado\Trabalho Final\Dados\Levantamento em Campo\Compiled.xlsx", sheet_name="Nix")
        self.pXRF = pd.read_excel(r"D:\Mestrado\Trabalho Final\Dados\Levantamento em Campo\Compiled.xlsx", sheet_name="pXRF")

        # Lendo Rasteres importantes
        self.talvegues = gpd.read_file(r"D:/Mestrado/Trabalho Final/SIG/HidrografiaArea.zip")

        # Lendo Rasteres
        print("Lendo Rasteres")
        self.uso_solo        = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/USOSOLO.tif")                 # Tipos de uso do solo
        self.elevation       = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/Elevation.tif")               # Elevação
        self.terrain_rug_idx = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/TerrainRuggednessIndex.tif")  # Variação de elevação entre um pixel e seus vizinhos imediatos
        self.topo_pos_idx    = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/TopograficPositionIndex.tif") # Elevação de um ponto com a média da elevação ao redor, topo, vale ou plano
        self.roughness       = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/Roughness.tif")               # A diferença entre a elevação máxima e mínima dentro de uma vizinhança
        self.slope           = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/Slope.tif")                   # Declividade
        self.aspect          = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/Aspect.tif")                  # Para onde "aponta" a face do terreno
        self.texture_02      = rxr.open_rasterio(r"D:\Mestrado\Trabalho Final\SIG\textura_2.tif")               # Textura a 2 cm
        self.texture_20      = rxr.open_rasterio(r"D:\Mestrado\Trabalho Final\SIG\textura_20.tif")              # Textura a 20 cm
        
        # X e Y dos valores
        if self.eval:
            xx, yy = np.meshgrid(self.uso_solo.x.values, self.uso_solo.y.values) # type: ignore
            self.x_y = np.column_stack([xx.ravel(), yy.ravel()])                 # type: ignore

        self.transformer = Transformer.from_crs("EPSG:31983", self.uso_solo.rio.crs, always_xy=True) # type: ignore
        self.transform = self.uso_solo.rio.transform() # type: ignore

        print("Processando dados")
        self._process_dados()

    def _process_dados(self):
        # Dados dos rasteres concatenados
        self.raster_data = np.array([
            self._norm_data(self.elevation.values[0]), # type: ignore
            self._norm_data(self.terrain_rug_idx.values[0]), # type: ignore
            self._norm_data(self.topo_pos_idx.values[0]), # type: ignore
            self._norm_data(self.roughness.values[0]), # type: ignore
            self._norm_data(self.slope.values[0]), # type: ignore
            self._norm_data(self.aspect.values[0]), # type: ignore
            self.uso_solo.values[0], # type: ignore
            self.texture_20.values[0], # type: ignore
        ])

        # Dados para convolução
        x, y = self.transformer.transform(self.dados["Lon"], self.dados["Lat"])
        row, col = rowcol(self.transform, x, y)

        jan = int((self.janela-1)/2)

        start_row = row-jan
        end_row   = row+jan

        start_col = col-jan
        end_col   = col+jan

        self.dados['s_row']=start_row
        self.dados['e_row']=end_row
        self.dados['s_col']=start_col
        self.dados['e_col']=end_col

        # Distância até o talvegue principal
        self.linhas_unidas = self.talvegues.union_all()
        pontos = gpd.GeoSeries(gpd.points_from_xy(self.dados["Lon"], self.dados["Lat"]), crs="EPSG:31983")
        dists = pontos.apply(lambda p: p.distance(self.linhas_unidas))
        self.dados["dist_talvegue"] = dists

    def _norm_data(self, data:np.ndarray)->np.ndarray:
        return (data - data.min())/(data.max()-data.min())

    def __len__(self):
        return self.uso_solo.size if self.eval else len(self.dados) # type: ignore

    def __getitem__(self, i):
        idx = i
        if isinstance(i, (int, float)):
            idx = [i]
        elif self.eval and (isinstance(i, slice) or isinstance(i, (list, np.ndarray))):
            if isinstance(i, slice):
                indices = list(range(*i.indices(len(self))))
            else:
                indices = i
            
            rasters = []
            values = []
            for idx in indices:
                raster, value = self.__getitem__(idx)
                rasters.append(raster[0])
                values.append(value[0])

            rasters = torch.stack(rasters)
            values = torch.stack(values)
            return rasters, values

        # Modo de gerar os dados finais
        if self.eval:
            x_y = self.x_y[i] # type: ignore
            x, y = self.transformer.transform(x_y[0], x_y[1])
            row, col = rowcol(self.transform, x, y)

            bands, height, width = self.raster_data.shape

            jan = int((self.janela-1)/2)
            window_shape = (bands, 2*jan + 1, 2*jan + 1)

            # Criar janela preenchida com NaN
            janela_data = np.full(window_shape, np.nan, dtype=self.raster_data.dtype)

            # Calcular limites válidos dentro do raster
            row_min = max(0, row - jan) # type: ignore
            row_max = min(height, row + jan + 1) # type: ignore

            col_min = max(0, col - jan) # type: ignore
            col_max = min(width, col + jan + 1) # type: ignore
            
            # Limites relativos à janela
            win_row_start = row_min - (row - jan)
            win_row_end   = win_row_start + (row_max - row_min)

            win_col_start = col_min - (col - jan)
            win_col_end   = win_col_start + (col_max - col_min)
            
            # Copiar a parte válida do raster para a janela
            janela_data[:, win_row_start:win_row_end, win_col_start:win_col_end] = self.raster_data[:, row_min:row_max, col_min:col_max]

            # Distância do talvegue
            p = Point(x, y)
            dist = torch.tensor(np.array([[distance(p, self.linhas_unidas)]]), device=self.device)

            rasters_vals = torch.tensor(np.array([janela_data]), device=self.device, dtype=torch.float64)

            return (rasters_vals, dist)

        # Pontos
        pontos = self.dados.loc[idx]

        # Valores nos rasteres
        rasters_vals = []
        for s_row, e_row, s_col, e_col in zip(pontos['s_row'], pontos['e_row'], pontos['s_col'], pontos['e_col']):
            rasters = np.array([])

            usos = []
            for uso_solo in self.columns_uso_solo:
                uso = np.where(self.raster_data[6, s_row:e_row+1, s_col:e_col+1]==uso_solo, 1, 0)
                usos.append(uso)
            usos = np.array(usos)

            tipos = []
            for idx, tipo_solo in enumerate(SOIL_TYPES):
                if tipo_solo is None:
                    continue
                tipo = np.where(self.raster_data[7, s_row:e_row+1, s_col:e_col+1]==idx, 1, 0)
                tipos.append(tipo)
            tipos = np.array(tipos)

            outras_bandas = self.raster_data[:6, s_row:e_row+1, s_col:e_col+1]
            raster_stack = np.concatenate([usos, tipos, outras_bandas], axis=0)
            rasters_vals.append(raster_stack)
        
        rasters_vals = torch.tensor(np.array(rasters_vals), device=self.device, dtype=torch.float64)

        # Demais dados
        K = torch.tensor(pontos["K (C1)"].values, device=self.device) * 1000 # K*1000 pois os valores estão baixos de mais
        dist_talvegue = torch.tensor(pontos[["dist_talvegue"]].values, device=self.device)

        if isinstance(i, (int, float)):
            dist_talvegue = dist_talvegue[0]
            rasters_vals = rasters_vals[0]
            K = K[0]


        return (rasters_vals, dist_talvegue), K
    
dataset = MeuDataset(device=device)

print("Len:", len(dataset))
dataset[15][0][0].shape, dataset[1:10][0][0].shape

Lendo Tabelas
Lendo Rasteres
Processando dados
Len: 83


(torch.Size([12, 15, 15]), torch.Size([10, 12, 15, 15]))

### Configurações do treino
---

- Seed para números aleatórios
- % de treino e teste
- Métricas
- Epochs
- Batch Size

In [ ]:
# Seed para permitir reprodutibilidade dos valores pseudo-aleatórios
seed = 42

# Porcentagens de Treino e Teste
train_percent = 85
test_percent  = 15

# BatchSize e Epochs
batch_size = 2
epochs     = 1000  # Poucos pontos, verificar overfitting

In [ ]:
n = len(dataset)
n_train = int(train_percent*n/100)
n_test = n - n_train
g = torch.Generator().manual_seed(seed)

train_ds, test_ds = random_split(dataset, [n_train, n_test], generator=g)

print("N Total:", n, "N Train:", n_train, "N Teste:", n_test)

train_ds[1:5][0][0].shape, test_ds[1:5][0][0].shape # type: ignore

### MLP e CNN
---

MLP configurada com uma CNN também

In [ ]:
# Modelo (MLP e CNN)
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = nn.Sequential(
            nn.Linear(34, 64),
            nn.Sigmoid(),
            nn.Linear(64, 64),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
        ).to(dtype=torch.float64)

    def forward(self, values):
        # x = self.conv(raster)
        # x = self.flatten(x)
        # x = torch.cat([x, values], dim=1)
        x = self.fc(values)
        return x

mlp = MLP().to(device=device)

X = dataset[1:4][0]
mlp(*X)

### Ajustar o modelo MLP

- Processos para ajustar a MLP pelo método do gradiente descendente

In [ ]:
def nse(y_pred, y_true, mean=None):
    if mean is not None:
        y_true_mean = mean
    else:
        y_true_mean = torch.mean(y_true)
        
    numerator = torch.sum((y_pred - y_true) ** 2)
    denominator = torch.sum((y_true - y_true_mean) ** 2)
    return 1 - (numerator / denominator)

In [ ]:
os.makedirs("best_model/", exist_ok=True)

# Média dos valores para compara erros
mean = dataset[:][1].mean()

model = MLP().to(device=device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = CyclicLR(
    optimizer,
    base_lr=0.0001,      # menor LR
    max_lr=0.1,          # maior LR
    step_size_up=100,    # número de iterações até atingir o max_lr
    mode='triangular',   # ou 'triangular2', 'exp_range'
    cycle_momentum=True  # necessário se usar Adam em vez de SGD
)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=batch_size)

best_test_loss = torch.inf
best_state = None

train_losses = []
val_losses = []

for epoch in range(1, epochs+1):
    # --- treino ---
    model.train()
    running_loss = 0.0
    for x, y in train_loader:

        optimizer.zero_grad()
        y_sim = model(*x)

        loss = criterion(y_sim[:, 0], y)
        running_loss += loss.item() * x[0].size(0)

        loss.backward()
        optimizer.step()
    
    train_loss = running_loss / len(train_loader.dataset) # type: ignore
    train_losses.append(train_loss)

    # --- validação ---
    model.eval()
    val_loss = 0.0
    nash = 0.0

    with torch.no_grad():
        for x, y in test_loader:
            # Forward
            y_sim = model(*x)

            loss = criterion(y_sim[:, 0], y)
            val_loss += loss.item() * x[0].size(0)

            ns = nse(y_sim[:, 0], y, mean)
            nash += ns.item() * x[0].size(0)

    # Média do loss
    val_loss /= len(test_loader.dataset) # type: ignore
    nash /= len(test_loader.dataset)     # type: ignore
    val_losses.append(val_loss)

    # early stopping simples
    current_lr = optimizer.param_groups[0]['lr']
    if val_loss < best_test_loss and epoch > 100:
        best_test_loss = val_loss
        best_state = {
            "lr":current_lr,
            "epoch": epoch,
            "epochs":epochs,
            "val_loss":val_loss,
            "train_loss":train_loss,
            "batch_size":batch_size,
            "model_state":model.state_dict(),
        }

        torch.save(best_state, f"best_model/{str(val_loss).replace(".", "_")}.pth")

    # Altero a taxa de aprendizado a cada fim da epoch para melhorar o aprendizado
    scheduler.step()

    #Print das métricas atuais

    indices = np.linspace(0, len(val_losses) - 1, 150, dtype=int)
    subset = [val_losses[i] for i in indices]

    ascii_chart = asciichartpy.plot(subset, {'height': 15})

    # Limpo o terminal
    clear_output(wait=True)
    print(f"Epoch {epoch}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Kc NASH: {(nash):.6f}cm/s | At lr:{current_lr:.4e}\n{ascii_chart}")
